In [1]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = 'NSE'

# Add `src` to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path
from pathlib import Path
if 'src' not in Path.cwd().parts:
    src_path = str(Path(get_path('purelib')) / '_src.pth')
    with open(src_path, 'w') as f:
        f.write(str(Path.cwd() / 'src\n'))

# Start the Jupyter loop
from ib_insync import util, LimitOrder, IB
util.startLoop()

In [3]:
# Set the root
from from_root import from_root
ROOT = from_root()

# Imports 
from utils import Vars, get_pickle
from loguru import logger

_vars = Vars(MARKET)
PORT = port = _vars.PORT
OPT_COLS = _vars.OPT_COLS[0]
DATAPATH = ROOT / 'data' / MARKET

# Straegy Check of `NSE`

In [5]:
df = get_pickle(DATAPATH / 'df_sow.pkl')

In [11]:
condition = (df.symbol == 'ALKEM') & (df.right == 'P')
df[condition][OPT_COLS]

,conId,symbol,undPrice,expiry,strike,right,dte,sdev,lot_size,margin,comm,ask,bid,last,optPrice,expPrice,prop,rom
17,674978772,ALKEM,5095.35,20240328,4050.0,P,45.24802,-1.371205,200.0,68277.05,20.0,-1.0,0.15,NaN,0.05,59.35,-0.829689,1.400029
